In [1]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

In [5]:

def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )

def logistic(x,r):
    return r*x*(1-x)


class logi(widgets.HBox):
     
    def __init__(self):
        super().__init__()
        output = widgets.Output()
 
        self.x = np.linspace(0, 1, 30)
        #initial_color = '#FF00DD'
        self.f=logistic
        lim=1.2
        with output:
            self.fig, self.ax = plt.subplots(constrained_layout=True, figsize=(5, 3.5))
            self.ax.set_xlim(0,lim)
            self.ax.set_ylim(0,lim)
        self.curve, = self.ax.plot(self.x, self.f(self.x,2))
        self.cobweb,= self.ax.plot([0],[0])
        self.bitect,= self.ax.plot([0,lim],[0,lim])

        self.fig.canvas.toolbar_position = 'bottom'
        self.nIterMax=30
        #self.ax.grid(True)
 
        # define widgets
        r_slider = widgets.FloatSlider(
            value=2, 
            min=0, 
            max=4, 
            continuous_update=True,
            description='r'
        )
        l_slider= widgets.FloatSlider(
            value=0, 
            min=0, 
            max=30, 
            continuous_update=True,
            description='l'
        )
        x_slider= widgets.FloatSlider(
            value=0.4, 
            min=0, 
            max=1, 
            continuous_update=True,
            description='x'
        )
        text_xlabel = widgets.Text(
            value='', 
            description='xlabel', 
            continuous_update=False
        )
        text_ylabel = widgets.Text(
            value='', 
            description='ylabel', 
            continuous_update=False
        )
 
        controls = widgets.VBox([
            r_slider, 
            l_slider,
            x_slider,
            text_xlabel, 
            text_ylabel
        ])
        controls.layout = make_box_layout()
         
        out_box = widgets.Box([output])
        output.layout = make_box_layout()
 
        # observe stuff
        widgets.interactive(self.updateCobwebs,r=r_slider, x=x_slider, l=l_slider ,continuous_update=True)
        r_slider.observe(self.updateCurve, 'value')
        text_xlabel.observe(self.update_xlabel, 'value')
        text_ylabel.observe(self.update_ylabel, 'value')
         
        text_xlabel.value = 'x'
        text_ylabel.value = 'y'
         
 
        # add to children
        self.children = [controls, output]

     
    def updateCurve(self, change):
        """Draw line in plot"""
        self.curve.set_ydata(self.f(self.x,change.new))
        self.fig.canvas.draw()
    
    def updateCobwebs(self,r,x,l):
        vertex=[[x],[x]]
        for i in range(self.nIterMax):  
            xp1=self.f(x,r)
            if l/np.abs(xp1-x)<1:
                vertex[0]+=[x]
                vertex[1]+=[x+l*np.sign(xp1-x)]
                break 
            if l/np.abs(xp1-x)<2:
                vertex[0]+=[x,x+l*np.sign(xp1-x)/2]
                vertex[1]+=[xp1,xp1]
                break
            
            vertex[0]+=[x,xp1]
            vertex[1]+=[xp1,xp1]
            l-=2*np.abs(x-xp1)
            x=xp1

        self.cobweb.set_xdata(vertex[0])
        self.cobweb.set_ydata(vertex[1])
        self.fig.canvas.draw()

    def update_xlabel(self, change):
        self.ax.set_xlabel(change.new)
 
    def update_ylabel(self, change):
        self.ax.set_ylabel(change.new)

    
         
         
logi()

logi(children=(VBox(children=(FloatSlider(value=2.0, description='r', max=4.0), FloatSlider(value=0.0, descrip…

In [19]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )
 
class Sines(widgets.HBox):
     
    def __init__(self):
        super().__init__()
        output = widgets.Output()
 
        self.x = np.linspace(0, 2 * np.pi, 100)
        initial_color = '#FF00DD'
 
        with output:
            self.fig, self.ax = plt.subplots(constrained_layout=True, figsize=(5, 3.5))
        self.line, = self.ax.plot(self.x, np.sin(self.x), initial_color)
         
        self.fig.canvas.toolbar_position = 'bottom'
 
        # define widgets
        int_slider = widgets.FloatSlider(
            value=1, 
            min=0, 
            max=10, 
            step=0.1, 
            description='freq'
        )
        amp_slider = widgets.FloatSlider(
            value=1, 
            min=0, 
            max=2, 
            step=0.02, 
            description='amp'
        )
        color_picker = widgets.ColorPicker(
            value=initial_color, 
            description='pick a color'
        )
        text_xlabel = widgets.Text(
            value='', 
            description='xlabel', 
            continuous_update=False
        )
        text_ylabel = widgets.Text(
            value='', 
            description='ylabel', 
            continuous_update=False
        )
 
        controls = widgets.VBox([
            int_slider, 
            amp_slider,
            color_picker, 
            text_xlabel, 
            text_ylabel
        ])
        controls.layout = make_box_layout()
         
        out_box = widgets.Box([output])
        output.layout = make_box_layout()
 
        # observe stuff
        widgets.interactive(self.update,change=int_slider,ampchange=amp_slider)
        color_picker.observe(self.line_color, 'value')
        text_xlabel.observe(self.update_xlabel, 'value')
        text_ylabel.observe(self.update_ylabel, 'value')
         
        text_xlabel.value = 'x'
        text_ylabel.value = 'y'
         
 
        # add to children
        self.children = [controls, output]
     
    def update(self, change, ampchange):
        """Draw line in plot"""
        self.line.set_ydata(ampchange*np.sin(change * self.x))
        self.fig.canvas.draw()
 
    def line_color(self, change):
        self.line.set_color(change.new)
 
    def update_xlabel(self, change):
        self.ax.set_xlabel(change.new)
 
    def update_ylabel(self, change):
        self.ax.set_ylabel(change.new)
         
         
Sines()

Sines(children=(VBox(children=(FloatSlider(value=1.0, description='freq', max=10.0), FloatSlider(value=1.0, de…